# Titrating, planar surface in explicit salt

This will simulate:

1. titrating species (COOH/COO-) allowed to move only on one box side
2. surrounding 1:1 salt treated with explicit ions in the GC ensemble.

![alternate text](mc.png)

### System Requirements
This Jupyter Notebook was originally run in MacOS 10.11 with `GCC 4.8`, `Python2`, `matplotlib`, `pandas` within the Anaconda environment. Contemporary Linux distributions such as Ubuntu 14.04 should work as well.

### Download and build Faunus

Wwe use a custom Metropolis Monte Carlo (MC) program build within the [Faunus framework](http://github.com/mlund/faunus.git). The sections below will fetch the complete faunus project and compile the program.

In [ ]:
from __future__ import division, unicode_literals, print_function
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np, pandas as pd
import os.path, os, sys, json
plt.rcParams.update({'font.size': 16, 'figure.figsize': [8.0, 6.0]})
try:
    workdir
except NameError:
    workdir=%pwd
else:
    %cd $workdir

In [44]:
%%bash -s "$workdir"
cd $1
if [ ! -d "faunus/" ]; then
    echo 'fau_example(surf "./" surf.cpp)' > mc/CMakeLists.txt
    git clone https://github.com/mlund/faunus.git
    cd faunus
    git checkout a15d197
else
    cd faunus
fi
pwd
CXX=c++ CC=cc cmake . -DCMAKE_BUILD_TYPE=RelWithDebInfo -DENABLE_APPROXMATH=on -DMYPLAYGROUND=`pwd`/../mc &>/dev/null
make surf -j4

/Users/mikael/github/faunus-notebooks/surface-titrating/faunus
[ 36%] Built target xdrfile
[ 81%] Built target libfaunus
Scanning dependencies of target surf
[ 90%] Building CXX object /Users/mikael/github/faunus-notebooks/surface-titrating/mc/CMakeFiles/surf.dir/surf.cpp.o
[100%] Linking CXX executable surf
[100%] Built target surf


### Create Input and run MC simulation

In [45]:
def mkinput():
    js = {
        "customlj" : {
            "Na COOH" : { "sigma":1.0, "eps":0.7 },
            "Na COO" : { "sigma":1.0, "eps":0.7 }
        },

        "processes": {
            "acid": { "pKd": 4.75, "pX": pH, "bound": "COOH", "free": "COO" }
            },

         "energy": {
             "eqstate": { "processfile": "gctit.json" },
             "nonbonded": {
                "coulomb": { "epsr": 80 },
                "ljsimple": { "eps": 0.05 }
                }
            },

         "system": {
             "temperature": 298.15,
             "cuboid": { "xyzlen": str(xylen)+' '+str(xylen)+' '+str(zlen) },
             "mcloop": { "macro": 10, "micro": micro }
            },

         "moves": {
             "gctit"         : { "molecule": "salt", "prob": 1.0 },
             "atomtranslate" : {
                 "lipid": { "prob": 1.0, "dir":"1 1 0" },
                 "salt":  { "prob": 1.0 }
                }
            },
        
         "moleculelist": {
              "lipid": { "Ninit": Nlipid, "atomic": True, "atoms": "COOH", "insdir": "1 1 0",
                        "insoffset":"0 0 "+str(surfpos) },
              "salt":  { "Ninit": 100, "atomic": True, "atoms": "Na Cl" }
            },

          "atomlist": {
                "Na":   { "q": 1,  "r": 2, "dp": 50, "eps":0.5, "activity": salt },
                "Cl":   { "q": -1, "r": 2, "dp": 50, "eps":0.5, "activity": salt },
                "COOH": { "q": 0,  "r": 2, "dp": 20, "eps":0.5 },
                "COO":  { "q": -1, "r": 2, "dp": 20, "eps":0.5 }
                }
          }

    with open('surf.json', 'w+') as f:
        f.write(json.dumps(js, indent=4))

%cd $workdir'/mc'

micro=50000
xylen=50
zlen=100
surfpos=-0.5*zlen+1e-6

for pH in [7.0]:
    for salt in [0.2]:     # mol/l
        for rho in [25]:  # angstrom^2 per lipid
            Nlipid = int(xylen**2/rho)
            print('Number of head groups = ', Nlipid)
            pfx='rho'+str(rho)+'-pH'+str(pH)+'-I'+str(salt)
            if True: #not os.path.isdir(pfx):
                %mkdir -p $pfx 
                %cd $pfx
                !rm -fR state
                mkinput()
                !../surf
                %cd ..

/Users/mikael/github/faunus-notebooks/surface-titrating/mc
Number of head groups =  100
/Users/mikael/github/faunus-notebooks/surface-titrating/mc/rho25-pH7.0-I0.2
# Reading process acid ... OK!
Reading space state file 'state'. State file not found.
# Reading process acid ... OK!

 .................
  Atom Properties  
 *****************
  Number of entries:       5
  Element info:
    unk   COO   COOH  Cl    Na    

 ...............................
  Simulation Space and Geometry  
 *******************************
  Boundary                 Cuboid (XY-periodicity)
  Volume                   250000 Å³ = 250 nm³ = 2.5e-22 liters
  Sidelengths               50  50 100 (Å)
  Scale directions         XYZ
  Number of particles      300
  Electroneutrality        Yes 0
  System sanity check      Passed
  Number of molecule types 2
  Groups:
    1     [0-99]           lipid       N/V = 0.0004 Å⁻³ = 664.216 mM  
    2     [100-299]        salt        N/V = 0.0008 Å⁻³ = 1328.43 mM  

 ........